In [1]:
from gensim.models import word2vec
import numpy as np
from TimeChecker import TimeChecker
import gensim
from CharToIndex import CharToIndex

In [2]:
tegaki_file_path = "/net/nfs2/export/home/ohno/CR_pytorch/data/tegaki_katsuji/tegaki_distance.npz"
katsuji_file_path = "/net/nfs2/export/home/ohno/CR_pytorch/data/tegaki_katsuji/katsuji_distance.npz"

tegaki_data = np.load(tegaki_file_path,allow_pickle=True)
katsuji_data = np.load(katsuji_file_path,allow_pickle=True)

d_val = np.concatenate((tegaki_data['value'],katsuji_data['value']))
d_ans = np.concatenate((tegaki_data['answer'],katsuji_data['answer']))
dataset=[]

In [3]:
# for val,ans in zip(d_val,d_ans):
#     ar = list(val+ans)
#     for i,item1 in enumerate(ar):
#         for j,item2 in enumerate(ar):
#             if item1 == item2:
#                 continue
#             dataset.append([item1,item2])

# np.save("/net/nfs2/export/home/ohno/CR_pytorch/Wrod2Vec/dataset.npy",dataset)
# running_time = TimeChecker()
# running_time.start()

# model = word2vec.Word2Vec(dataset,vector_size=64,min_count=1,window=2, epochs=100,sg=1,compute_loss=True)


# print('実行時間： ',running_time.stop())

# #モデルの保存
# model.save("/net/nfs2/export/home/ohno/CR_pytorch/Wrod2Vec/word2vec.model")

In [11]:

# dataset = np.load("/net/nfs2/export/home/ohno/CR_pytorch/Wrod2Vec/dataset.npy",allow_pickle=True)
path = r"/net/nfs2/export/home/ohno/CR_pytorch/Wrod2Vec/context_2/"
# model_64 = gensim.models.Word2Vec.load(path+"word2vec_64.model")
# model_128 = gensim.models.Word2Vec.load(path+"word2vec_128.model")
# model_256 = gensim.models.Word2Vec.load(path+"word2vec_256.model")
# model_512 = gensim.models.Word2Vec.load(path+"word2vec_512.model")
# model_1024 = gensim.models.Word2Vec.load(path+"word2vec_1024.model")

model = gensim.models.Word2Vec.load(path+"word2vec_1024.model")




In [5]:
import pprint
def most_similaries(char):
    pprint.pprint(model_64.wv.most_similar(char))
    pprint.pprint(model_128.wv.most_similar(char))
    pprint.pprint(model_256.wv.most_similar(char))
    pprint.pprint(model_512.wv.most_similar(char))
    pprint.pprint(model_1024.wv.most_similar(char))




In [6]:
print(model_64.get_latest_training_loss())
print(model_128.get_latest_training_loss())
print(model_256.get_latest_training_loss())
print(model_512.get_latest_training_loss())
print(model_1024.get_latest_training_loss())


134217728.0
134217728.0
134217728.0
134217728.0
134217728.0


In [8]:
chars_file_path = r"/net/nfs2/export/home/ohno/CR_pytorch/data/tegaki_katsuji/all_chars_3815.npy"
tokens = CharToIndex(chars_file_path)

In [13]:
import torch

vocab_num = len(tokens)
vector_size = 1024

attention_matrix = torch.zeros((vocab_num,vector_size),dtype=float)

for i,char in enumerate(tokens.table):
    try:
        attention_matrix[i] = torch.tensor(model.wv[char.decode()])
    except KeyError:
        attention_matrix[i] = 0
        print(char.decode())

<UNK>
　
〜
埃
媚
寵
樽
璧
瞑
舐
蹴
：
ａ
ｂ
ｃ
ｄ
ｅ
ｆ
ｈ
ｉ
ｋ
ｎ
ｐ
ｒ
ｓ
ｕ
ｘ


In [33]:
c = '人'
vector =  torch.mul(torch.tensor(model.wv[c],dtype=float),torch.ones(attention_matrix.shape))

cos_sims = torch.nn.functional.cosine_similarity(attention_matrix,vector,dim=1)
softmax = torch.nn.Softmax(dim=0)
attentioned_value = softmax(cos_sims)

infer = torch.topk(torch.sigmoid(cos_sims),20)
for sim,id in zip(infer[0],infer[1]):
    print(tokens.get_decoded_char(id.item()),end=' ')
    print(sim)

人 tensor(0.7311, dtype=torch.float64)
入 tensor(0.6113, dtype=torch.float64)
八 tensor(0.6068, dtype=torch.float64)
へ tensor(0.5963, dtype=torch.float64)
火 tensor(0.5945, dtype=torch.float64)
大 tensor(0.5926, dtype=torch.float64)
べ tensor(0.5914, dtype=torch.float64)
バ tensor(0.5913, dtype=torch.float64)
ベ tensor(0.5887, dtype=torch.float64)
卜 tensor(0.5845, dtype=torch.float64)
ん tensor(0.5839, dtype=torch.float64)
パ tensor(0.5830, dtype=torch.float64)
Ｋ tensor(0.5818, dtype=torch.float64)
ヘ tensor(0.5794, dtype=torch.float64)
ハ tensor(0.5786, dtype=torch.float64)
穴 tensor(0.5785, dtype=torch.float64)
尺 tensor(0.5776, dtype=torch.float64)
ペ tensor(0.5765, dtype=torch.float64)
ト tensor(0.5746, dtype=torch.float64)
く tensor(0.5728, dtype=torch.float64)


In [ ]:

print(torch.norm(attention_matrix,dim=1).shape,torch.norm(vector).shape)
attention = torch.mv(attention_matrix,vector) / torch.norm(attention_matrix,dim=1) * torch.norm(vector)
print(model_512.wv.most_similar(c))

softmax = torch.nn.Softmax(dim=0)
attentioned_value = softmax(attention)
infer = torch.topk(attentioned_value,20)
for sim,id in zip(infer[0],infer[1]):
    print(tokens.get_decoded_char(id.item()),end=' ')
    print(sim)

In [ ]:
import random
index = int(len(d_val)*random.random())
for item in d_val[index:index+100]:
    print(''.join(item))

（しトＦ卜Ｔ１ドレＩ
機臓磯械礒倣微概横艤
れ礼札机社壮杠丸ね肛
）コ３－ヨョノＪユュ
がぶびガかぴボ尤ぷバ
日曰目ロ白月口臼貝自
本木水奉太未汰末氷禾
にヒ仁ん比穴しル父七
ぉおむねだあぁ沿右わ
けげ什サＨは叶仕汁ケ
るろゐ弓乃６万石Ｏ乙
差圭着基菫茎垂墓淫
別則副刑馴剛剖割削剃
の①めＤ⑥０わゎぬ②
原源厚頂便須県項煩盾
点小ＡムＨ以ねれみ４
な女か左交びだ友ぴひ
の①め⑥Ｄ０ぬ②わゎ
でてさ万芯す下セむせ
すず寸十ナオォサザ女
がぶびかガぴボ尤ぷバ
、ーゝ－一Ｘ１入Ｙ）
美実芙業支失夫案笑矢
はほぼばぱぽな仕住け
こごニ二と乙Ｃさ三ヒ
の①め⑥０Ｄ②わゎぬ
感惑威憾戚蔵球成毬懲
覚党筧寛堂覧賞宜意貞
はほぼばぱぽな仕ぬけ
現規硯親視蜆覗坦境理
代伐伏化什氏供杙仇仕
人入八ハヘバ火大太へ
も屯むち宅丸おぉ古忘
持特将接蒋待侍痔符
っつづうぅウゥろら。
てで七モ亡せこセマズ
ぃいＵにぴりびひじ゛
ょよま主上決土注王士
すず十寸ナザオォサ女
。．゜のＯＣ・０６゛
例倒側俐制剖捌働何創
ぇえ之穴文ネス乏元丈
ばぱぼぽげほ伏吠伐獄
、ゝ－Ｘ入）へ一ーベ
葬華溝萎準革蕪蒸幸
式武氏弐比丈北大犬攻
のめ①⑥ぬ０Ｄわゎり
ぁあめおぉ交訪も窃女
とどヒをさ２Ｅ七上そ
に仁ヒん穴し比Ｋ七化
塩堆堪雄垢搗推臨蝋
をき守安そな壱巻逮ま
振娠板坂張抜掘披握根
るろゐ弓乃６石万Ｏ乙
とさヒを七き上ミ２よ
ぃいＵにぴびりひじ゛
ぅうウゥらろラづつっ
行斤竹付方庁右待打存
偽為慈活弟浩鴻持蔦鳥
はほばぼぱぽ仕な住体
ょよま主上注土決王士
さきざまよょ吉ミを古
に仁穴ヒ化欠忙Ｒた比
ＴＦトし１Ｉ卜（エェ
機磯臓械穣微横捗倣徴
れわゎ肛札礼机ね抗丸
リコ）りヨョゴ゛ＪＵ
をき守ぞな安そ巷壱
ＦＴ下ド（５ＶＥじＰ
祓赦滅誠城械威減祗戒
ぅうウゥラらろづつっ
コリ）り゛ゴヨョＵ・
行斤付庁竹升什方打待
偽為弟慈詩活蔦鴻浩鳥
そぞ号弓モをで名て壱
の①めぬ⑥いぃわゎ０
も屯むち毛６しら宅忘
の①め⑥ぬ②０Ｄわゎ
でてさ芯む万やゃマた
すず寸ナ十女オォサ才
。゜０Ｃロ６Ｄ・ら．
まよょ主注決土浜洪来
ただ九えぇ之犬七大穴
、ゝー－一Ｘ入）Ｙベ
子孑干予千不平チヂ乎
供倶僕位棋併俳仇佼
ただえぇ九之七犬大穴
ちぢおぉらも右ろ方だ
がびぶかガぴボぷおぉ
ょよま上おぉ主ぶ土ぷ
くぐ（べぺベペこミ丈
ゃやヤャ多